# Exercises Week 12 - Association Rule Mining

In [2]:
import numpy as np

# Local imports
import sys
sys.path.append('../utilities')
from load_data import load_market_basket

## Exercise 1.

Consider a dataset of transactions $D$, and let $D'$ be a dataset derived from $D$ by independently deleting items from transactions in $D$. In particular, any item in any transaction in $D$ is deleted with probability $p$.

1. Given an itemset $S$, compute its expected support in $D'$ as a function of its support in $D$.
2. Compute the probability that an itemset $S$, which is frequent in $D$, is also frequent in $D'$, under the same minimum support threshold.

### Solutions
1.  To solve this, we need to think of what the probability is for loosing each "supporting" transaction.
    Suppose that $|S| = k$ and consider a transaction $t_i$ of which $S \subseteq i(t_i)$, then since each item in $t_i$ is deleted independently with probability $p$, the probability of keeping the support from $t_i$ is $Pr[\text{not deleting any item } x\in S \text{ from }t_i] = \prod_{x \in S} (1-p) = (1-p)^k$.  
    Thus computing the expected value of the support for $S$ in $D'$ amounts to
    $$
        \mathbb{E}\left[Sup(S, D')\right] = \sum_{t_i \in D , S \subset t_i} (1-p)^k 1 = (1-p)^k Sup(S, D)
    $$
    
2.  To compute this probability, we again consider an itemset $S$ with size $k$, i.e., $|S| = k$.
    Again, we can realize that a transaction $t_i$ is going to be supportive to $S$ if all $x \in S$ of $t_i$ are kept. This has probability $\hat p = (1-p)^k$.
    We can think of $\hat p$ as a bernouli distribution.  
    
    Now we need to quantify the probability of keeping at least the minimum support ($m$) of the supporting transactions. 
    Each of the $n := Sup(S, D)$ supporting transactions will be kept with probability $\hat p$ so we can think of this setup as a binomial distribution with $n$ trials.
    The binomial distribution has probability mass function $Pr[X = k] = {n \choose k }\hat p^k(1-\hat p)^{n-k}$ for bernouli trials with $n$ samples (supportive transactions in this case) and exactly $k$ positive outcomes.
    
    Now we just need to sum up the probabilities of all $k \geq m$.
    
    $$
        Pr[sup(S, D') \geq m] = \sum_{k=m}^{n} {n \choose k}\hat p^k (1-\hat p)^{n - k}
    $$

## Exercise 2.

We have learned how to mine frequent itemsets and association rules from a transaction database where each transaction consists of a simple set of items. You are asked to propose a framework for mining association rules from transaction data, in which each item in a transaction is associated with an integer number that counts how many times the items appears in the transaction. In a market basket, this count number indicates the number of copies of a product in a customer’s basket. For example, we do not only care whether a customer bought fish or not, but how many pieces of fish they bought.

1. Define (extend) the notion of an itemset and an association rule in the case of such data.

### Solution:
1. 
    1. The simple solution would be to let the "identifiers" be the item and the count. This would give rules like [(A, 2), (C, 3) => (D, 1)].
    2. One could also keep the original formulation of an itemset but change the count by counting as many supports as the min count of the items in each transaction.
    3. Finally, one could compute mean counts in supporting transactions, i.e., [A:2, B:3.5, D:1.2] and if I then make a rule say "E => ABD", We could then give an additional information on how many items one could expect.

## Exercise 3.

The work you do in this exercise will be very useful also in your hand-in.

We learned the Apriori algorithm in class. Make your own implementation. 

We will use the anonymized real-world retail market basket data from: http://fimi.ua.ac.be/data/.

This data comes from an anonymous Belgian retail store, and was donated by Tom Brijs from Limburgs Universitair Centrum, Belgium. The original data contains 16,470 different items and 88,162 transactions. You may only work with the top-50 items in terms of occurrence frequency.

Your task is to:
1. Implement the Apriori algorithm.
2. Apply the Apriori algorithm on these data to find association rules with minimum support 0.05 and minimum confidence 0.7. Write down the rules in descending order of confidence.


In [5]:
# Load the retail data
transactions = load_market_basket()

def filter_transactions(T, k=50):
    """
        Keep only the top k items in the transactions.
        Remove transactions that become empty.
    """
    # Count occurences of each item
    counts = [0] * 16470
    for t in T:
        for i in t:
            counts[i] += 1

    # Sort and select top k
    counts = np.array(counts)
    order  = np.argsort(counts)[::-1] # reverse the sorted order

    indexes_to_keep = order[:k]       # Keep the top k items
    index_set = set(indexes_to_keep)  # Convert to python set for efficiency

    # Filter transactions
    T_new = [t_ for t_ in  [list(filter(lambda i: i in index_set, t)) for t in T]  if t_]
    return T_new

T = filter_transactions(transactions, k=50)

[[9], [32], [36, 38, 39, 41], [38, 39, 48], [38, 39, 48, 49]]
[[9], [32], [36, 38, 39, 41], [38, 39, 48], [38, 39, 48, 49]]


In [ ]:
# Tiny function for generating rules from tuples
# Ex: rule((1, 2), (5)) outputs "(1, 2) => (5)"
rule  = lambda lhs, rhs: "%s => %s" % (str(lhs), str(rhs)) # For generating rule strings

def apriori_algorithm(T, support=0.05, min_confidence=0.7):
    """
        Apriori algorithm for mining association rules.
        Inputs:
            T:               A list of lists, each inner list will contiain integer-item-ids. 
                             Example: T = [[1, 2, 5], [2, 3, 4], [1, 6]]
            support:         The proportion of occurences needed to keep itemsets.
            min_confidence:  Minimum confidence for the algorithm to output the rule.
        
        Outputs:
            rules:           List of tuples [(rule:str, confidence:float), ... ]
                             Example: [("(1, 2) => (5)", 0.84), ("(3, 4) => (7)", 0.75)]
    """
    rules = []
    ### TODO Your code here
    
    
    
    ### TODO Your code here
    return rules

Below you can try your algorithm. If you want, you could try changing the support and min_confidence to see what happens to the result.

In [ ]:
rules = apriori_algorithm(T, support=0.05, min_confidence=0.7)
rules = sorted(rules, key=lambda x: x[1], reverse=True)

print("%-8s \t %s" % ("Conf.", "Rule"))
for r in rules:
    print("%7.4f%% \t %s" % r[::-1])

## (OPTIONAL) Exercise 4.

This exercise is a continuation of exercise 2. 

1. Describe an efficient algorithm that mines itemsets and association rules as defined in Exercise 2. Illustrate the pruning strategies used in your algorithm and explain how they relate to the Apriori principle.

2. Extend your implementation of the Apriori algorithm to handle this case.